![image_1770101639028.png](./image_1770101639028.png "image_1770101639028.png")

In [0]:
%sql
CREATE TABLE if not exists employee_login (
    emp_id INT,
    login_date DATE
)
USING DELTA;
INSERT INTO employee_login VALUES
(101, '2024-01-01'),
(101, '2024-01-02'),
(101, '2024-01-03'),
(102, '2024-01-01'),
(102, '2024-01-03'),
(102, '2024-01-04'),
(103, '2024-01-10'),
(103, '2024-01-11'),
(103, '2024-01-12'),
(103, '2024-01-13'),
(103, '2024-01-13');


In [0]:
%sql
with tbl1 as (select emp_id, datediff(login_date, lag(login_date,1) over(partition by emp_id order by login_date)) as diff from employee_login)
select emp_id from tbl1 group by emp_id having max(diff) in (1,0)

![image_1770101735923.png](./image_1770101735923.png "image_1770101735923.png")

In [0]:
%sql
CREATE TABLE employee_hierarchy (
    emp_id INT,
    emp_name STRING,
    manager_id INT
)
USING DELTA;
INSERT INTO employee_hierarchy VALUES
(1, 'CEO', NULL),
(2, 'CTO', 1),
(3, 'CFO', 1),
(4, 'Manager1', 2),
(5, 'Manager2', 2),
(6, 'Lead1', 4),
(7, 'Engineer', 6);


In [0]:
%sql
select * from employee_hierarchy

In [0]:
%sql
select *, dense_rank() over( order by manager_id asc) as level from employee_hierarchy;

![image_1770102438953.png](./image_1770102438953.png "image_1770102438953.png")

In [0]:
%sql
CREATE TABLE match_results (
    match_id INT,
    team1 STRING,
    team2 STRING,
    winner STRING,
    result STRING,
    match_date DATE
)
USING DELTA;
INSERT INTO match_results VALUES
(1, 'IND', 'AUS', 'IND', 'WIN',  '2024-01-01'),
(2, 'IND', 'ENG', NULL,  'DRAW', '2024-01-03'),
(3, 'IND', 'SA',  'IND', 'WIN',  '2024-01-05'),
(4, 'AUS', 'ENG', 'AUS', 'WIN',  '2024-01-02'),
(5, 'AUS', 'SA',  NULL,  'DRAW', '2024-01-04'),
(6, 'ENG', 'SA',  'ENG', 'WIN',  '2024-01-06'),
(7, 'IND', 'NZ',  'NZ',  'WIN',  '2024-01-07'),
(8, 'PAK', 'SL',  NULL,  'DRAW', '2024-01-08');


In [0]:
%sql
with 
teams as
(select match_id,team1,winner,result from match_results union 
select match_id,team2,winner,result from match_results),
count as 
(select count(team1) as played,team1 from teams group by team1),
results as 
(select *, case when team1=winner then 1 else 0 end as wins,
case when winner is null then 1 else 0 end as draws,
case when team1!=winner and result= 'WIN' then 1 else 0 end as losses
from teams ),
points as(
select team1 as team, sum(wins) as wins, sum(draws) as draws,sum(losses) as losses,sum(wins)*2+sum(draws) as points from results group by team1)
select t1.team1 as team,t1.played, t2.wins,t2.draws,t2.losses,t2.points from count t1 join points t2 on t1.team1=t2.team order by points desc


![image_1770104698108.png](./image_1770104698108.png "image_1770104698108.png")

In [0]:
%sql
CREATE TABLE students (
    id INT,
    student STRING
)
USING DELTA;
INSERT INTO students VALUES
(1, 'A'),
(2, 'B'),
(3, 'C'),
(4, 'D'),
(5, 'E');


In [0]:
%sql
select id,student from students order by id

In [0]:
%sql
select id,
case 
  when id%2==1 then 
    case 
      when lead(student) over(order by id) is not null then lead(student) over(order by id) 
      else student 
    end
  when id%2==0 then lag(student) over(order by id)
end as student
from students;

--or

select id,
case 
  when id%2==1 then lead(student,1,student) over(order by id)
  when id%2==0 then lag(student) over(order by id)
end as student
from students


![image_1770106145950.png](./image_1770106145950.png "image_1770106145950.png")

In [0]:
%sql
CREATE TABLE journeys (
    customer_id INT,
    from_city STRING,
    to_city STRING,
    journey_order INT
)
USING DELTA;
INSERT INTO journeys VALUES
(1, 'Delhi',     'Hyderabad',  1),
(1, 'Hyderabad', 'Chennai',    2),
(1, 'Chennai',   'Coimbatore', 3),
(2, 'Mumbai',    'Pune',       1),
(2, 'Pune',      'Bangalore',  2);


In [0]:
%sql
    SELECT
        customer_id,
        from_city,
        to_city,
        journey_order,
        ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY journey_order) AS rn_start,
        ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY journey_order DESC) AS rn_end
    FROM journeys

In [0]:
%sql
WITH ranked_journey AS (
    SELECT
        customer_id,
        from_city,
        to_city,
        journey_order,
        ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY journey_order) AS rn_start,
        ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY journey_order DESC) AS rn_end
    FROM journeys
)
SELECT
    customer_id,
    MAX(CASE WHEN rn_start = 1 THEN from_city END) AS from_city,
    MAX(CASE WHEN rn_end = 1 THEN to_city END) AS to_city
FROM ranked_journey
GROUP BY customer_id
ORDER BY customer_id;
